In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.05672540302024105, Any[sparse([5], [1], [0.47634190670249055], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(30,30), rand(30,30))

function model(X)
    return ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
end
println(ctx.Params[1].val)


[0.23188318455064327 0.5891637878708811 0.6983017161315782 0.029879751528545162 0.9375005956256776 0.8934897776364834 0.1866305690165605 0.13353961291983174 0.20012242606260122 0.9582777219230499 0.38747651081262235 0.3949844229128775 0.46135076591945656 0.6707398474365539 0.8897225773865806 0.4904294832723134 0.8565772181631001 0.05356047880198733 0.6026677363763757 0.9071696203113021 0.4838959157008679 0.2768783309784749 0.5697687864376787 0.9727024226952268 0.5253486593802865 0.08184604228378434 0.2324863280696724 0.010658321588975284 0.26511519414944107 0.24489465607590455; 0.33887045132968086 0.9821406883170236 0.6320509453331445 0.04632943990984206 0.45619570494726625 0.922820054538308 0.805793018440791 0.9211747710404722 0.34519127083731604 0.34718233485978933 0.6077918661125598 0.8583468466580151 0.8634126081992086 0.7003742165265527 0.9568831368257786 0.5894533851741971 0.7823126483382099 0.43813080795238857 0.7411978840507198 0.8782807554320907 0.3914170245411476 0.6347699132

 0.7186388960322763 0.06347150682877567 0.16541097035856434 0.0691025323685337 0.26409545675166746 0.9206411220315415 0.3281406217790197 0.5988798639961194 0.5739312922390415; 0.08421185975639955 0.5194267924787999 0.21924943136861363 0.07722419258903612 0.8600829649196629 0.678556207173886 0.40114797591453977 0.14677252984641287 0.6625351371949711 0.29731321012462064 0.35163252471228057 0.9764644370454157 0.19509684762619728 0.4605489502643014 0.7715088353534973 0.5070991367777379 0.840463657633863 0.14591877327301384 0.19338731236140805 0.8480758061723499 0.8286774453063984 0.7856734802361175 0.5944058666646568 0.38118328440771676 0.30021805125218015 0.492032405508675 0.6123297434519994 0.3236295929783448 0.856552048524372 0.5880857018792609; 0.9109815587733756 0.10665271158271916 0.15409962061650984 0.29051141189058316 0.029421074943000947 0.8292664665140126 0.6801405168472413 0.11458275663332151 0.7435346448447592 0.40144148075984487 0.7944966498172157 0.1051332404235722 0.81581122

7.176348740478039e7


21.378827117944258
22.37451719037764
21.52812705257398
22.806865240637457
19.65671098374182
21.788718286558403
22.92088325798107
19.913552589731243
20.48600573688735
22.490475821470483
21.75841855204089
21.48885886270364
22.260294642929882
20.878891500534596
23.27491477604712
20.951924483245865
23.017367286875
22.54600537329144
22.955809509393877
20.12683190351312
22.80957858030538
19.471612510146638
22.083248100663752
21.01593381812371
20.773731544606864
23.61890346687849
22.36693328223045
20.253225284490867
23.80204966788758
21.796519927136544
21.15901110507906
21.34433950663021
22.08205271109791
19.064222961627397
23.494194522502145
21.06641149958996
19.873262979186247
23.32199113087761
22.228085256532122
24.430563451978596
19.339711391941645
20.028366792244313
19.95350141875852
19.825506845904187
21.70388980678677
21.44119615958866
22.303657739925903
25.474020458273376
23.38693973427372
24.51984915695495
22.066053278006144
20.765404792098913
21.95878576700933
22.19470462953362
21.9

21.345266904891282
22.543574359303975
21.58780357068995
22.343026617097586
20.347192122294288
23.034094832284847
21.31268181452736
19.774561038694006
20.608387499526117
19.78882622911347
21.91964714918561
21.545079017149504
23.568132923689703
20.730275631061883
22.474175616919293
22.447012620433313
22.09320921789266
21.05736612419685
20.351106747681822
22.720687088284897
23.72848109814105
21.75229737303912
22.33848283349869
21.991354113705153
18.721300768757423
23.11754815481043
21.59410756748866
22.299182918789764
22.47241535574101
21.99037675673272
19.94593210181833
20.84162101720586
21.583278776415113
22.396368115305638
21.521869091723296
22.389641429365415
20.275821784422458
21.67368346928457
21.673024063320604
21.22368136599376
21.421498851368348
21.947876522045323
20.63596112187919
21.200123042673237
22.965751099290184
22.13807910043466
21.394116810428383
23.85359441308611
21.764220340100277
21.531301586725707
22.760870901502788
22.386491400904056
22.702323293455116
23.1441757594

21.93617324253809
21.012774754467962
22.686722264643404
24.373363539165172
23.274822387513503
20.327821788665876
21.7198360304382
21.775602687226918
24.198219074010726
22.032433740246585
21.709460667402052
20.140916834406845
19.771373248231804
21.398861849678262
21.486650871137865
21.35760937273563
22.305718109764474
22.540741280233604
22.028279158904052
21.180108416499554
21.464667190318185
22.887667472799905
20.840669807161127
23.888698598845046
21.421672386525636
22.141056076134582
21.783597599203077
22.684602092770362
22.39400581271855
21.401866491044213
22.466789796330133
23.7925685462555
20.25748328799176
22.19558947899053
22.862996023749943
22.349071034582064
21.009371163199418
21.357310889445934
23.328093876168555
21.565578147423455
19.827807453223983
21.661277816140647
21.48840289906317
19.790262812193134
20.24217272787429
19.407159903434064
20.331207477336907
21.877448545553726
23.227082636831902
21.271569539333395
21.9100321425782
21.741781524824574
20.537690056037537
21.889

23.503157558180934
21.47266686444429
22.188026509655124
22.373271646365467
22.218298168939896
21.620729930522906
21.071344224672366
22.20320171823663
21.28165319913223
22.62147904708846
24.350713999263196
19.997603905505976
21.47517345024539
22.118352411865345
20.460231559697593
21.53614433947075
21.731261181789506
20.22220598417845
21.393461459872032
21.584000496725842
21.383444717435957
22.209907401617077
22.290227830234112
20.337091521234733
22.445532109091516
22.77976157088266
23.082866045784588
20.409032623048876
22.33280069242517
22.150382785498284
21.12518910031073
21.374646034197834
22.71955033557419
22.29681379143181
22.238859488456708
22.415415750519713
20.422171962706614
21.74657356249176
21.81893617709925
22.00162385291263
20.34747254249039
21.026337181625966
21.380290853790665
22.678280577039626
25.088302430666698
22.6765265687855
21.652864033414
21.408129953005187
23.002828749210128
21.877780978844857
23.498518035144034
23.228121914615013
22.72644986122427
21.106834842921

; -6.529160637029119e7 -1.0006094163321683e7 -1.687916392767151e7 -7.67645767919313e7 -2.351998312082454e7 -3.4665201957964756e7 -1.2371419476046277e7 -5.322271386065484e7 -7.861222048348565e7 -1.8924442787266057e7 -7.442027799120869e7 -4.907293703961e7 -7.71504928296417e6 -8.032877172166244e6 -4.055907089302498e6 -2.5470520297922343e7 -2.7374547561594367e7 -7.343066370714341e7 -7.458788293956756e6 -2.3981103155251373e7 -1.4663570759312976e7 -2.3231243770891264e7 -3.81210474036708e7 -5.1259584308509335e7 -5.127061312225304e7 -6.714447308547159e7 -1.1235541154664608e7 -4.550283594127047e7 -4.241274352005964e7 -7.103613562158439e7; -5.203871502515406e7 -7.975057468943691e6 -1.345303093956083e7 -6.118290129046463e7 -1.8745896042250857e7 -2.762885877110363e7 -9.860269149985813e6 -4.241956637019973e7 -6.265551020576601e7 -1.5083158429068329e7 -5.931444909869601e7 -3.91121115705682e7 -6.149048652209087e6 -6.402363226950183e6 -3.232638893984788e6 -2.03005138565431e7 -2.1818061389141403e7 -5.8

 -1.2597880218852898e7 -2.1251217190113436e7 -9.664819230682741e7 -2.9612145857863814e7 -4.364420743509963e7 -1.5575874161633136e7 -6.7008499030717045e7 -9.897441367701091e7 -2.382626509683089e7 -9.36966713380786e7 -6.17838434039595e7 -9.713407955017399e6 -1.011355859800813e7 -5.106470083318247e6 -3.206791237494889e7 -3.4465122608646624e7 -9.245072650123043e7 -9.390769273900677e6 -3.0192705662996463e7 -1.8461740218729008e7 -2.9248617481113955e7 -4.79951883124496e7 -6.453687811994534e7 -6.455076431640323e7 -8.453628199004604e7 -1.4145778911207177e7 -5.728901298781854e7 -5.33985230736168e7 -8.943596634719755e7; -5.327014727693019e7 -8.163777438534816e6 -1.377138049369361e7 -6.263071933361392e7 -1.918949511649143e7 -2.8282661202765554e7 -1.0093600170812406e7 -4.3423372853474714e7 -6.413817547970854e7 -1.5440082804240752e7 -6.071805181674959e7 -4.0037651312201895e7 -6.294558393771872e6 -6.553868016536859e6 -3.3091345258217724e6 -2.07809000246532e7 -2.2334358361287e7 -5.9910644622857854e7 -